# Classification de noms en utilisant un RNN

Dans ce tutorie nous introduisont les réseaux de neurones récurrents (RNNs).

Pour cet fin, nous travaillerons sur la classification de noms qui peuvent être soient une compagnie ou une personne. 

In [1]:
import math
import time
import torch
import random
import copy
import numpy as np
from torch import optim, nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split

In [2]:
from torchvision.utils import make_grid

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
cuda_device = 3
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")
batch_size = 32
learning_rate = 0.01
n_epoch = 30

La taille du jeu de données ici est très petite et sert seulement à démontrer le concept

In [4]:
with open('names_manual_annotation.tsv') as f:
    data = f.readlines()
    data = [d[:-1].split('\t') for d in data]
data = [(x, y) for x, y in data if y != 'Unknown']

In [5]:
data[:10]

[('Québec (Ville de)', 'Company'),
 ('Drouin', 'Person'),
 ('Pneus Métro inc.', 'Company'),
 ('Graton', 'Person'),
 ('English', 'Person'),
 ('Clinique vétérinaire Ève Woods-Lavoie inc.', 'Company'),
 ('Grimard', 'Person'),
 ('St-Amand', 'Person'),
 ('Douville', 'Person'),
 ('Fréchette', 'Person')]

In [6]:
len(data)

793

### Nous allons travailler lettre par lettre

In [7]:
def format_examples(data):
    examples = list()
    for name, tag in data:
        examples.append((list(name), tag))
    return examples

In [8]:
formatted_data = format_examples(data)
formatted_data[1]

(['D', 'r', 'o', 'u', 'i', 'n'], 'Person')

### train test split

In [9]:
formatted_data = random.sample(formatted_data, len(formatted_data))
train_ratio = int(len(formatted_data)*0.8) # 80% of dataset
train = formatted_data[:train_ratio]
test = formatted_data[train_ratio:]
valid_ratio = int(len(train)*0.8) # 20% of train set
valid = train[valid_ratio:]

In [10]:
len(train), len(valid), len(test)

(634, 127, 159)

Nous devons nous créer un vocabulaire pour toute donnée "non numérique". Ce vocabulaire nous sert d'index qui sera utilisé pour trouvé le vecteur de rééls associé à cet élément non numérique.

In [12]:
vocab = set()
tags = set()

for example in train:
    for char in example[0]:
        vocab.add(char)
    tags.add(example[1])
    
char_to_idx = {
    '<PAD>': 0,
    '<UNK>': 1,
}

for char in sorted(vocab):
    char_to_idx[char] = len(char_to_idx)
    
tag_to_idx = {tag: i for i, tag in enumerate(sorted(tags))}

In [13]:
char_to_idx

{'<PAD>': 0,
 '<UNK>': 1,
 ' ': 2,
 '&': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '/': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'X': 44,
 'Y': 45,
 'Z': 46,
 'a': 47,
 'b': 48,
 'c': 49,
 'd': 50,
 'e': 51,
 'f': 52,
 'g': 53,
 'h': 54,
 'i': 55,
 'j': 56,
 'k': 57,
 'l': 58,
 'm': 59,
 'n': 60,
 'o': 61,
 'p': 62,
 'q': 63,
 'r': 64,
 's': 65,
 't': 66,
 'u': 67,
 'v': 68,
 'w': 69,
 'x': 70,
 'y': 71,
 'z': 72,
 'É': 73,
 'â': 74,
 'ç': 75,
 'è': 76,
 'é': 77,
 'ê': 78,
 'ë': 79,
 'î': 80,
 'ô': 81,
 '—': 82}

In [14]:
tag_to_idx

{'Company': 0, 'Person': 1}

Le Vectorizer va nous servir à convertir toute donnée 'non numérique' en donnée numérique.

In [15]:
class Vectorizer:
    def __init__(self, char_to_idx, tag_to_idx):
        self.char_to_idx = char_to_idx
        self.tag_to_idx = tag_to_idx

    def vectorize_sequence(self, sequence, idx, remove_if_unk=False):
        if '<UNK>' in idx:
            unknown_index = idx['<UNK>']
            chars = [idx.get(tok, unknown_index) for tok in sequence]
            if remove_if_unk:
                return [w for w in chars if w != unknown_index]
            else:
                return chars

        else:
            return [idx[tok] for tok in sequence]

    def __call__(self, example):
        name, tag = example
        vectorized_name = self.vectorize_sequence(name, self.char_to_idx)
        vectorized_tag = self.tag_to_idx[tag]
        return (
            vectorized_name,
            vectorized_tag,
        )

vectorizer = Vectorizer(char_to_idx, tag_to_idx)

In [16]:
train_data = [vectorizer(example) for example in train]
valid_data = [vectorizer(example) for example in valid]
test_data = [vectorizer(example) for example in test]

datasets = {'train':train_data, 'val':valid_data, 'test':test_data}

In [18]:
train_data[0], train[0]

(([32, 47, 62, 61, 55, 60, 66, 51], 1),
 (['L', 'a', 'p', 'o', 'i', 'n', 't', 'e'], 'Person'))

Le concept de padding est extrêmement important. Il nous permet d'envoyer des tenseurs de longueurs différentes sur le GPU.

Nous prenons donc le tenseur le plus long de notre minibatch pour créer une matrice d'exemple.

In [19]:
import torch

def pad_sequences(vectorized_seqs, seq_lengths):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(seq[:seqlen])
    return seq_tensor

def collate_examples(samples):
    names, tags = list(zip(*samples))
    names_lengths = torch.LongTensor([len(s) for s in names])
    padded_names = pad_sequences(names, names_lengths)
    tags = torch.LongTensor(tags)
    return padded_names, tags

In [20]:
from torch.utils.data import DataLoader, Dataset

batch_size = 16

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    collate_fn=collate_examples,
    shuffle=True
)

valid_loader = DataLoader(
    valid_data,
    batch_size=batch_size,
    collate_fn=collate_examples,
    shuffle=False
)

test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    collate_fn=collate_examples,
    shuffle=False
)

In [21]:
b = next(iter(train_loader))
b[0].shape, b

(torch.Size([16, 48]),
 (tensor([[39, 55, 60, 53, 58, 51, 66, 61, 60,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
          [38, 55, 61, 67, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
          [38, 51, 65, 66, 47, 67, 64, 47, 60, 66,  8, 22, 47, 64,  2, 32, 47,  2,
           38, 61, 49, 54, 51, 58, 55, 76, 64, 51,  2, 55, 60, 49,  9,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
          [25, 70, 49, 47, 68, 47, 66, 55, 61, 60, 65,  2, 27,  9,  2, 21, 58, 58,
           47, 64, 50,  2, 55, 60, 49,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
          [20, 13, 13, 11,  8, 17, 17, 15, 19,  2, 37, 67, 77, 48, 51, 49

In [24]:
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

class NameClassifier(nn.Module):
    def __init__(self, char_to_idx, tag_to_idx, embedding_size, hidden_layer_size):
        super(NameClassifier, self).__init__()
        self.embeddings = nn.Embedding(len(char_to_idx), embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_layer_size)
        self.fully_connected = nn.Linear(hidden_layer_size, len(tag_to_idx))
        self.loss_function = nn.CrossEntropyLoss()
        self.metrics = ['acc']

    def forward(self, names):
        # Getting the length of sequences so we can pack them and send them to gpu.
        # We also sort the sequences by length, as required by the pack_padded_sequence function
        seq_lengths, perm_idx = (names > 0).sum(dim=1).sort(0, descending=True)
        
        # We need the reverse idx to unsort the sequence at the end of the forward
        _, rev_perm_idx = perm_idx.sort(0)
        
        # (batch_size, max_length)
        sorted_names = names[perm_idx]
        
        # (batch_size, max_length, embedding_size)
        embeds = self.embeddings(sorted_names)
        
        packed_names = pack_padded_sequence(embeds, seq_lengths, batch_first=True)
        
        # (1, batch_size, hidden_layer_size)
        _, (h_n, _) = self.rnn(packed_names)
        
        # (1, batch_size, num_tags)
        out = self.fully_connected(h_n)
        
        # (batch_size, num_tags)
        out = out.squeeze(0)
        
        return out[rev_perm_idx]

In [25]:
dataloaders = {'train':train_loader, 'val':valid_loader, 'test':test_loader}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val', 'test']}

In [26]:
from torch.optim import lr_scheduler

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [29]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [31]:
net = NameClassifier(char_to_idx, tag_to_idx, embedding_size=50, hidden_layer_size=100)
net = net.to(device)

In [32]:
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
loss_function = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, 1, gamma=1) # no scheduling
train_model(net, loss_function, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.5859 Acc: 0.7587
val Loss: 0.4102 Acc: 0.9134

Epoch 1/24
----------
train Loss: 0.3497 Acc: 0.9069
val Loss: 0.2655 Acc: 0.9370

Epoch 2/24
----------
train Loss: 0.2755 Acc: 0.9132
val Loss: 0.2310 Acc: 0.9449

Epoch 3/24
----------
train Loss: 0.2547 Acc: 0.9180
val Loss: 0.2126 Acc: 0.9449

Epoch 4/24
----------
train Loss: 0.2445 Acc: 0.9211
val Loss: 0.2054 Acc: 0.9449

Epoch 5/24
----------
train Loss: 0.2357 Acc: 0.9211
val Loss: 0.1978 Acc: 0.9449

Epoch 6/24
----------
train Loss: 0.2281 Acc: 0.9243
val Loss: 0.1868 Acc: 0.9449

Epoch 7/24
----------
train Loss: 0.2170 Acc: 0.9243
val Loss: 0.1857 Acc: 0.9449

Epoch 8/24
----------
train Loss: 0.2081 Acc: 0.9290
val Loss: 0.1741 Acc: 0.9528

Epoch 9/24
----------
train Loss: 0.1974 Acc: 0.9306
val Loss: 0.1633 Acc: 0.9528

Epoch 10/24
----------
train Loss: 0.1822 Acc: 0.9385
val Loss: 0.1337 Acc: 0.9528

Epoch 11/24
----------
train Loss: 0.1519 Acc: 0.9479
val Loss: 0.0889 Acc: 0.9843

Ep

NameClassifier(
  (embeddings): Embedding(83, 50)
  (rnn): LSTM(50, 100)
  (fully_connected): Linear(in_features=100, out_features=2, bias=True)
  (loss_function): CrossEntropyLoss()
)

In [33]:
net.cpu()
idx_to_char = {v:k for k, v in char_to_idx.items()}
idx_to_tag = {v:k for k,v in tag_to_idx.items()}
predictions = []
for inputs, labels in dataloaders['test']:
    seq = list(map(lambda seq : ''.join([idx_to_char[int(c)] for c in seq if 'PAD' not in idx_to_char[int(c)]]), inputs) )
    pred = [idx_to_tag[int(l)] for l in torch.argmax(net(inputs),dim=1)]
    label =[idx_to_tag[int(l)] for l in labels]
    predictions.extend(zip(seq, pred, label))
predictions

[('Frégeau', 'Person', 'Person'),
 ('Wesler', 'Person', 'Person'),
 ('Bordeleau', 'Person', 'Person'),
 ('Fils Transport inc.', 'Company', 'Company'),
 ('Tremblay', 'Person', 'Person'),
 ('Bérubé', 'Person', 'Person'),
 ('9308-5934 Québec inc. (Volvo Trois-Rivières)', 'Company', 'Company'),
 ('Comptois', 'Person', 'Person'),
 ('Khan', 'Person', 'Person'),
 ('DIM Stone inc.', 'Company', 'Company'),
 ('Mazda Canada inc.', 'Company', 'Company'),
 ('Ferme Les 2B inc.', 'Company', 'Company'),
 ('Martineau', 'Person', 'Person'),
 ('Parent (Maison de rêve PGV)', 'Company', 'Company'),
 ('Javid', 'Person', 'Person'),
 ('Verhoef', 'Person', 'Person'),
 ('Moore', 'Person', 'Person'),
 ('Clinique vétérinaire <UNK>ve Woods-Lavoie inc.', 'Company', 'Company'),
 ('Dubé', 'Person', 'Person'),
 ('Gheys Allah', 'Person', 'Person'),
 ('Brochu', 'Person', 'Person'),
 ('CHUM Hôpital Notre-Dame', 'Company', 'Company'),
 ('Nathan (Custom Rides)', 'Company', 'Company'),
 ('Giasson', 'Person', 'Person'),
 ('P